# Analysis of matched RX and TX rates across valid ranges

This notebook analyzes the link status of samples rates from 100 MHz to 500 MHz, where both the TX and RX rates at the framers are the same. This is primarily looking for issues and/or patherns with the clocking configurations to determine if there are issues with JIF or another component in the system.

To scale the sample rate from 100 MHz to 500 MHz, decimators, interpolators, ADC rate, and DAC rate are automatically determine to be within valid ranges.

Details of the system:
- Hardware: AD9081 + ZCU102
- Linux master branch from 3/23/2022
- HDL master branch from 3/23/2022
- VCXO 122.88 MHz
- TX JESD Mode 9, RX JESD Mode 10.00

In [1]:
import json
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
#!pip install pyqt5
%matplotlib inline

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.graph_objects as go
init_notebook_mode(connected=True)

## Import and filter data to extract desired log fields

In [2]:
files = glob.glob("logs/ad9081_100_1_500_04122022/*.json")

results = []
for file in files:
    with open(file) as f:
        data_ref = json.load(f)
        data = {}
        if 'cfg' not in data_ref:
            continue
        if 'jdevices_statuses' not in data_ref:
            continue
        data['status'] = data_ref['status']
        data['rx_lane_rate'] = data_ref['cfg']['jesd_adc']['bit_clock']
        data['tx_lane_rate'] = data_ref['cfg']['jesd_dac']['bit_clock']
        data['rx_sample_rate'] = data_ref['cfg']['jesd_adc']['sample_clock']
        data['tx_sample_rate'] = data_ref['cfg']['jesd_dac']['sample_clock']
        data['DAC_freq'] = data_ref['param_set']['DAC_freq']
        for k in data_ref['param_set']:
            data[k] = data_ref['param_set'][k]

        ks = data_ref['jdevices_statuses'].keys()
        data['rx_link_status'] = data_ref['jdevices_statuses']['84a90000.axi-jesd204-rx']['Link status'] == "DATA"
        data['tx_link_status'] = data_ref['jdevices_statuses']['84b90000.axi-jesd204-tx']['Link status'] == "DATA"
        data['rx_link_state'] = data_ref['jdevices_statuses']['84a90000.axi-jesd204-rx']['Link status']
        data['tx_link_state'] = data_ref['jdevices_statuses']['84b90000.axi-jesd204-tx']['Link status']
        # Merge clock dividers
        od = data_ref['cfg']['clock']['out_dividers']
        od.sort()
        ods = ''
        for i, k in enumerate(od):
            ods += f" / {k}"

        data[f"out_dividers"] = ods[2:]
            
        if 'jdevices_statuses' in data_ref:
            ks = list(data_ref['jdevices_statuses'].keys())
            data[ks[0]] = data_ref['jdevices_statuses'][ks[0]]['enabled']
            data[ks[1]] = data_ref['jdevices_statuses'][ks[1]]['enabled']
    results.append(data)

# print(results[0].keys())
df = pd.DataFrame(results, columns=results[0].keys())
df.sort_values(by=['status'])

,status,rx_lane_rate,tx_lane_rate,rx_sample_rate,tx_sample_rate,DAC_freq,ADC_freq,fddc,cddc,fduc,cduc,rx_link_status,tx_link_status,rx_link_state,tx_link_state,out_dividers,84a90000.axi-jesd204-rx,84b90000.axi-jesd204-tx
0,failed,1.400000e+10,1.400000e+10,350000000,350000000,5600000000,2800000000,4,2,8,2,False,True,CGS,DATA,4 / 4 / 5 / 256 / 256,enabled,enabled
113,failed,7.280000e+09,7.280000e+09,182000000,182000000,2912000000,1456000000,4,2,8,2,False,True,CGS,DATA,4 / 5 / 8 / 512 / 512,enabled,enabled
112,failed,6.640000e+09,6.640000e+09,166000000,166000000,3984000000,1992000000,6,2,6,4,False,True,CGS,DATA,4 / 5 / 8 / 512 / 512,enabled,enabled
105,failed,7.440000e+09,7.440000e+09,186000000,186000000,2976000000,1488000000,4,2,8,2,False,True,CGS,DATA,4 / 5 / 8 / 512 / 512,enabled,enabled
103,failed,1.428000e+10,1.428000e+10,357000000,357000000,5712000000,2856000000,4,2,8,2,False,True,CGS,DATA,4 / 4 / 5 / 256 / 256,enabled,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,passed,1.352000e+10,1.352000e+10,338000000,338000000,5408000000,2704000000,4,2,8,2,True,True,DATA,DATA,4 / 4 / 5 / 256 / 256,enabled,enabled
33,passed,5.680000e+09,5.680000e+09,142000000,142000000,3408000000,1704000000,6,2,6,4,True,True,DATA,DATA,4 / 10 / 50 / 640 / 640,enabled,enabled
35,passed,1.136000e+10,1.136000e+10,284000000,284000000,4544000000,2272000000,4,2,8,2,True,True,DATA,DATA,4 / 5 / 50 / 320 / 320,enabled,enabled
126,passed,5.920000e+09,5.920000e+09,148000000,148000000,3552000000,1776000000,6,2,6,4,True,True,DATA,DATA,4 / 10 / 50 / 640 / 640,enabled,enabled


## Plots of Link Enabled Pass/Fail at Boot

In [3]:
# Plot pass/fail over frequency
results = df[['status','rx_sample_rate','ADC_freq','DAC_freq','rx_link_status','tx_link_status','rx_link_state','tx_link_state']]
results_ref = results.copy()
## TX
pc = pd.DataFrame([True]*len(results['status']),columns=['passed'])
choices = [1,0]
conditions = [results['tx_link_status'] == pc['passed'],results['tx_link_status'] != pc['passed']]

out = list(np.select(conditions, choices))
results_tx = results.assign(passed = out);
results_tx.sort_values(by=['rx_sample_rate'], inplace=True)
## RX
results = results_ref.copy()
pc = pd.DataFrame([True]*len(results['status']),columns=['passed'])
choices = [1,0]
conditions = [results['rx_link_status'] == pc['passed'],results['rx_link_status'] != pc['passed']]

out = list(np.select(conditions, choices))
results_rx = results.assign(passed = out);
results_rx.sort_values(by=['rx_sample_rate'], inplace=True)


In [4]:
trace0 = Scatter(
  x=results_tx['rx_sample_rate'],
  y=results_tx['passed'],
  name='TX LINK'
)
trace1 = Scatter(
  x=results_rx['rx_sample_rate'],
  y=results_rx['passed'],
  name='RX LINK'
)
fig = go.Figure()
fig.add_trace(trace0)
fig.add_trace(trace1)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [0,1],
        ticktext = ['Failed','Passed']
    ),
    xaxis = dict(title='RX Sample Rate')
)
iplot(fig)

In [5]:
## STATES
results_rx = results_ref.copy()
results_rx.sort_values(by=['rx_sample_rate'], inplace=True)

In [6]:
trace0 = Scatter(
  x=results_tx['rx_sample_rate'],
  y=results_tx['tx_link_state'],
  name='TX LINK STATE'
)
trace1 = Scatter(
  x=results_rx['rx_sample_rate'],
  y=results_rx['rx_link_state'],
  name='RX LINK STATE'
)
fig = go.Figure()
fig.add_trace(trace0)
fig.add_trace(trace1)
fig.update_layout(
    xaxis = dict(title='RX Sample Rate'),
    yaxis = dict(title='Link State Post Boot')
)
iplot(fig)